In [1]:
import cv2
import numpy as np
import os
import keras
from keras.models import Sequential, Model,load_model
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from keras.initializers import glorot_uniform

In [61]:
def identity_block(X, f, filters, stage, block):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters

    X_shortcut = X
   
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X = Add()([X, X_shortcut])# SKIP Connection
    X = Activation('relu')(X)

    return X

In [62]:
def convolutional_block(X, f, filters, stage, block, s=2):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [107]:
def ResNet50(input_shape=(28, 28, 3)):

    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)

    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')


    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    X = X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
    
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

In [108]:
base_model = ResNet50(input_shape=(28, 28, 3))

In [109]:
headModel = base_model.output
headModel = Flatten()(headModel)
headModel=Dense(256, activation='relu', name='fc1',kernel_initializer=glorot_uniform(seed=0))(headModel)
headModel=Dense(128, activation='relu', name='fc2',kernel_initializer=glorot_uniform(seed=0))(headModel)
headModel = Dense( 1,activation='sigmoid', name='fc3',kernel_initializer=glorot_uniform(seed=0))(headModel)

In [110]:
model = Model(inputs=base_model.input, outputs=headModel)

In [111]:
model.summary()


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 28, 28, 3)]  0           []                               
                                                                                                  
 zero_padding2d_4 (ZeroPadding2  (None, 34, 34, 3)   0           ['input_10[0][0]']               
 D)                                                                                               
                                                                                                  
 conv1 (Conv2D)                 (None, 14, 14, 64)   9472        ['zero_padding2d_4[0][0]']       
                                                                                                  
 bn_conv1 (BatchNormalization)  (None, 14, 14, 64)   256         ['conv1[0][0]']            

# **Download RESNET50 default weights**

In [ ]:
# for layer in base_model.layers:
#     layer.trainable = False

In [112]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [113]:
from keras.datasets import mnist
(X_train,y_train),(X_test,y_test) = mnist.load_data()
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((60000, 28, 28), (10000, 28, 28), (60000,), (10000,))

In [114]:

X_train = X_train[59000::]
X_test = X_test[9000::]
y_train = y_train[59000::]
y_test = y_test[9000::]
X_train.shape,X_test.shape,y_train.shape,y_test.shape


((1000, 28, 28), (1000, 28, 28), (1000,), (1000,))

In [115]:
X_train = np.stack((X_train,)*3, axis=-1)
X_test = np.stack((X_test,)*3, axis=-1)
# X_train = X_train.repeat(8, axis=1).repeat(8, axis=2)
# X_test = X_test.repeat(8, axis=1).repeat(8, axis=2)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((1000, 28, 28, 3), (1000, 28, 28, 3), (1000,), (1000,))

In [116]:

X_train = X_train/255
X_test = X_test/255

In [123]:
model.fit(X_train, y_train, epochs = 10, batch_size = 32)  

Epoch 1/10
32/32 [==============================] - 3s 75ms/step - loss: nan - accuracy: 0.0980
Epoch 2/10
32/32 [==============================] - 1s 42ms/step - loss: nan - accuracy: 0.0980
Epoch 3/10
32/32 [==============================] - 1s 41ms/step - loss: nan - accuracy: 0.0980
Epoch 4/10
32/32 [==============================] - 1s 42ms/step - loss: nan - accuracy: 0.0980
Epoch 5/10
32/32 [==============================] - 1s 41ms/step - loss: nan - accuracy: 0.0980
Epoch 6/10
32/32 [==============================] - 1s 41ms/step - loss: nan - accuracy: 0.0980
Epoch 7/10
32/32 [==============================] - 1s 41ms/step - loss: nan - accuracy: 0.0980
Epoch 8/10
32/32 [==============================] - 1s 42ms/step - loss: nan - accuracy: 0.0980
Epoch 9/10
32/32 [==============================] - 2s 48ms/step - loss: nan - accuracy: 0.0980
Epoch 10/10
32/32 [==============================] - 2s 53ms/step - loss: nan - accuracy: 0.0980


In [118]:
import tensorflow
from tensorflow.keras.applications.resnet50 import ResNet50
model1 = Sequential()

model1.add(ResNet50(include_top = False, pooling = 'avg', weights = 'imagenet'))
model1.add(Flatten())
model1.add(Dense(256,activation='relu'))
model1.add(Dense(10,activation='softmax'))
model1.layers[0].trainable = False

In [119]:
model1.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [122]:
model1.fit(X_train, y_train, epochs = 10, batch_size = 32)  

Epoch 1/10
32/32 [==============================] - 1s 26ms/step - loss: 0.4495 - accuracy: 0.8770
Epoch 2/10
32/32 [==============================] - 1s 18ms/step - loss: 0.3974 - accuracy: 0.8830
Epoch 3/10
32/32 [==============================] - 1s 17ms/step - loss: 0.3786 - accuracy: 0.8900
Epoch 4/10
32/32 [==============================] - 1s 16ms/step - loss: 0.3671 - accuracy: 0.8940
Epoch 5/10
32/32 [==============================] - 1s 17ms/step - loss: 0.3394 - accuracy: 0.8990
Epoch 6/10
32/32 [==============================] - 0s 13ms/step - loss: 0.3315 - accuracy: 0.9050
Epoch 7/10
32/32 [==============================] - 0s 13ms/step - loss: 0.2907 - accuracy: 0.9230
Epoch 8/10
32/32 [==============================] - 0s 13ms/step - loss: 0.2867 - accuracy: 0.9110
Epoch 9/10
32/32 [==============================] - 0s 12ms/step - loss: 0.2572 - accuracy: 0.9310
Epoch 10/10
32/32 [==============================] - 0s 13ms/step - loss: 0.2488 - accuracy: 0.9330
